### Ref.: datacamp

In [1]:
import findspark
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('haveFun').getOrCreate()

In [2]:
spark

In [3]:
spark.version

'2.1.0'